# Parametric Koopman Learning Tests on Duffing Oscillator

In [ ]:

import sys
sys.path.append('..')
import PKoopmanDL as pkdl
import numpy as np
import matplotlib.pyplot as plt
import torch

# Define ODE
ode = pkdl.DuffingOscillator()

# Define flow map
dt = 1e-3
t_step = 0.25
flowmap = pkdl.ForwardEuler(t_step, dt)

# Generate data set
n_traj = 5
traj_len = 10
x_min = -1
x_max = 1
param_min = -1
param_max = 1
dataset = pkdl.ParamODEDataSet(ode, flowmap)
dataset.generate_data(n_traj, traj_len, x_min, x_max, param_min, param_max)

# build dictionary
dim_output = 25
dim_nontrain = 3
hidden_layer_sizes = [100, 100, 100]
nontrain_func = lambda x: torch.cat([torch.ones((x.size(0), 1)).to(x.device), x], dim=1)
dictionary_network = pkdl.FullConnResNet(ode.dim, dim_output - dim_nontrain, hidden_layer_sizes)
dictionary = pkdl.TrainableDictionary(dictionary_network, nontrain_func, ode.dim, dim_output, dim_nontrain)

# build solver
n_epochs = 30
batch_size = 5
reg = 0.1
reg_final = 0.01
tol = 1e-6
lr = 1e-4
koopman_network = pkdl.FullConnResNet(ode.param_dim, dim_output**2, hidden_layer_sizes)
PK = pkdl.ParamKoopman(dim_output, koopman_network)
solver = pkdl.ParamKoopmanDLSolver(dictionary)
PK = solver.solve(dataset, PK, n_epochs, batch_size, tol, lr, lr)

CUDA is available. GPU is being used.


Training:   0%|          | 0/30 [00:00<?, ?it/s]

5
torch.Size([5, 625])
torch.Size([5, 25, 25])
torch.Size([5, 25])
